# Day 19
## Puzzle 1

In [904]:
import numpy as np
from scipy import optimize as opt
from itertools import combinations
import networkx as nx
import re
from tqdm import tqdm

In [905]:
input_file = 'input_1.txt'
# input_file = 'test_input_1.txt'

Read towel patterns and designs from input.

In [906]:
with open(file=input_file, mode="r") as file:
    text_parts = file.read().split('\n\n')
    patterns = text_parts[0].replace(' ', '').split(',')
    designs = text_parts[1].split('\n')

In [907]:
patterns[:10]

['urbu',
 'wrrbwrg',
 'rgug',
 'uwb',
 'uwg',
 'wubwgu',
 'bgrwu',
 'gubbuu',
 'wrrub',
 'rbr']

In [908]:
designs[:10]

['wbrurgggrgbuwguwuwuwuwwwwgugwbwrwwuguwrrwubwuwrgwbugw',
 'uubrgurbrbgwgbbrruwrwrwgbrrgguuwrwbrgwgwgrgrgu',
 'bugwgurruguwwbgwuuggrrbrubgggbgrubbwuuuuuurrgwwurwguwugggg',
 'brwugubrrbrgubrwubgwbrggrrrbwburgbugrwbrwubbg',
 'gbggwbrwugrbbrrbbrgbbbuguwwurrgguwrbubbbgbbgbbbg',
 'bgrwgugurwugurrwwgwbrruubuwuwggrubwrbbugggw',
 'wbrwrugwbugrubgwuggguwubbbrbrgrugwugggggrbbugwbgbbrbbg',
 'rwrbbrubwrbbbwurggwgurrwubgrwwwurwgburbbwgwu',
 'rubbbgggwuwggubuwgrrwurwgwbrwrguurugbgrgbbuburgbg',
 'bggubbgrgruubbbubrbburggguwgggrrguuwuubrugbgbgwuruur']

Turn matching patterns into column vectors of the matrix $A$ and use MILP (mixed integer linear programming) to solve the equation $A\vec{x}=\vec{b}$, where $\vec{b}=\begin{pmatrix} 1 & 1 & 1 & ... & 1 \end{pmatrix}^{T}$, for $x_i \in \{0, 1\}$. Every feasible solutions counts towards the total number of possible towel designs.

In [909]:
num_possible_designs = 0

for design in tqdm(designs):
    design_num_characters = len(design)
    vector_matches = []

    for pattern in patterns:
        for found_match in re.finditer(f'(?={re.escape(pattern)})', design):
            match_start = found_match.start()
            match_end = match_start + len(pattern)
            vector_match = [0]*match_start + [1]*len(pattern) + [0]*(design_num_characters - match_end)
            vector_matches.append(vector_match)

    A = np.matrix(vector_matches).T
    b = np.array([1]*design_num_characters)
    c = np.array([0]*A.shape[1])
    bounds = opt.Bounds(0, 1)
    integrality = np.full_like(c, True)
    constraints = opt.LinearConstraint(A, lb=b, ub=b)
    result = opt.milp(c=c, constraints=constraints, integrality=integrality, bounds=bounds)
    
    if result.success:
        num_possible_designs += 1

100%|██████████| 400/400 [00:02<00:00, 141.12it/s]


In [910]:
num_possible_designs

300

## Puzzle 2

Once again, create nodes and edges in order to create directed graphs and count the number of walks from the start nodes, to the end nodes, for every towel design.

In [911]:
num_possible_designs = 0

for design in tqdm(designs):
    design_length = len(design)
    nodes = []
    start_nodes = []
    end_nodes = []
    edges = []

    for pattern in patterns:
        for found_match in re.finditer(f'(?={re.escape(pattern)})', design):
            match_start = found_match.start()
            match_end = match_start + len(pattern)
            node = (match_start, match_end)
            nodes.append(node)

            if node[0] == 0:
                start_nodes.append(node)

            elif node[1] == design_length:
                end_nodes.append(node)
    
    for n1, n2 in combinations(nodes, 2):
        if n1[1] == n2[0]:
            edges.append((n1, n2))

        elif n2[1] == n1[0]:
            edges.append((n2, n1))

    D = nx.DiGraph()
    D.add_edges_from(edges)

    if not D.nodes:
        continue

    start_nodes = [start_node for start_node in start_nodes if start_node in D.nodes]
    end_nodes = [end_node for end_node in end_nodes if end_node in D.nodes]
    start_nodes_index = np.array([node in start_nodes for node in D.nodes])
    end_nodes_index = np.array([node in end_nodes for node in D.nodes])
    A = nx.linalg.adjacency_matrix(D).toarray()
    current_rows = A[start_nodes_index, :]
    
    while current_rows.sum() > 0:
        num_possible_designs += int(current_rows[:, end_nodes_index].sum())
        current_rows = current_rows@A

100%|██████████| 400/400 [00:07<00:00, 56.87it/s]


In [912]:
num_possible_designs

624802218898092